## OpenStreetMap - Cincinnati - Data Wrangling Project

OpenStreetMap (OSM) is a collaborative project to create a free editable map of the world.Map data is collected from scratch by volunteers performing systematic ground surveys using tools such as a handheld GPS unit, a notebook, digital camera, or a voice recorder. The data is then entered into the OpenStreetMap database.

This data can be obtained in two formats: Extensible Markup Language (XML) and Protocol Buffer Binary Format (PBF). I am going to use XML format for this project.

The map area that I will be working on this project is the city of Cincinnati, Ohio. The data was obtained using 'MapZen - Metro Extracts'. The size of the data is approximately 198 MB.

Since this data is user generated and multiple users can contribute simultaneously, there is a possibility that, the data might have some inconsistencies. The aim of this project is to apply data wrangling techniques to:
1. Audit the dataset to find any inconsistencies
2. Clean the dataset and write the cleaned data to a file
3. Use the files created to load the data into a SQL Server Database
4. Extract insights from the database

## 1. Auditing the Data

I perform this step to find any inconsistencies in the data, so that I can address them and bring represent them in a consistent format.

I first start out finding the total number of users that have contributed to the dataset.

In [1]:
import xml.etree.cElementTree as ET

def get_user_id(element):
    return element.attrib["uid"]

def get_users(filename):
    users = set()
    
    for _, element in ET.iterparse(filename):
        if element.tag == "node" or element.tag == "way" or element.tag == "relation":
            user = get_user_id(element)
            users.add(user)
    return users

filename = 'cincinnati.osm'
users = get_users(filename)

print('Total number of users: ', len(users))

Total number of users:  531


I then try to find street names that are incosistent or different from generic street names. Inconsistent street names can include names written with abbreviations or names that have special characters in them or other generic street names that are missing from my expected list of street names. The code below helps me find such street names.

In [2]:
from collections import defaultdict
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected_street_names = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Place', 'Square', 'Lane', 'Road', 'Trail', 'Parkway', 'Commons', 'Way', 'Pike']

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected_street_names:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == 'addr:street')

def audit(osmfile):
    osm_file = open(osmfile, 'r', encoding='utf8')
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events = ('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

street_types = audit(filename)

print(street_types)

defaultdict(<class 'set'>, {'ter': {'firshade ter'}, 'Rd': {'Bridgetown Rd', 'Salem Rd'}, 'avenue': {'Hamilton avenue'}, 'Highway': {'Dixie Highway'}, 'St': {'Chickasaw St', 'Clark St', 'E 7th St', 'W Fifth St', 'Greenup St', '215 Calhoun St', 'Vine St'}, 'Ln': {'Fox Cub Ln'}, 'Ave': {'Kenard Ave', 'Erie Ave', 'Whetsel Ave', 'W Clifton Ave', 'Harrison Ave'}, 'Pkwy': {'Valley Plaza Pkwy'}, 'Plaza': {'Procter & Gamble Plaza'}, 'Circle': {'Mount Adams Circle'}, 'Hill': {'Liberty Hill'}, 'State': {'Ritchie State'}, 'Warner': {'Warner'}, 'Rd.': {'Montgomery Rd.'}, 'Crossing': {'North Bend Crossing'}, '508': {'508'}, 'Avnue': {'South Fort Thomas Avnue'}, 'Ludlow': {'Ludlow'}, 'Pass': {'Bremen Pass'}})


Output from the above code:

defaultdict(<class 'set'>, {'Pike': {'Wooster Pike', 'Highland Pike', 'Springfield Pike', 'Cleves Warsaw Pike', 'Alexandria Pike', 'Buttermilk Pike'}, 'ter': {'firshade ter'}, 'Rd': {'Bridgetown Rd', 'Salem Rd'}, 'Way': {'Albert Sabin Way', 'Campus Way', 'Levee Way', 'Distillery Way', 'Aquarium Way', 'East Mehring Way', 'Clock Tower Way', 'West Pete Rose Way'}, 'avenue': {'Hamilton avenue'}, 'Highway': {'Dixie Highway'}, 'St': {'E 7th St', 'Clark St', '215 Calhoun St', 'Greenup St', 'W Fifth St', 'Vine St', 'Chickasaw St'}, 'Ln': {'Fox Cub Ln'}, 'Ave': {'W Clifton Ave', 'Harrison Ave', 'Kenard Ave', 'Whetsel Ave', 'Erie Ave'}, 'Pkwy': {'Valley Plaza Pkwy'}, 'Plaza': {'Procter & Gamble Plaza'}, 'Circle': {'Mount Adams Circle'}, 'Hill': {'Liberty Hill'}, 'State': {'Ritchie State'}, 'Warner': {'Warner'}, 'Rd.': {'Montgomery Rd.'}, 'Crossing': {'North Bend Crossing'}, '508': {'508'}, 'Avnue': {'South Fort Thomas Avnue'}, 'Ludlow': {'Ludlow'}, 'Pass': {'Bremen Pass'}})

Based on this output, I created a mapping of inconsistent street names and used it to update street names while inserting them in a csv file. The mapping and the code to update the street names can be found below.

In [3]:
mapping = {
    "St": "Street",
    "St.": "Street",
    "Rd": "Road",
    "Rd.": "Road",
    "Ave": "Avenue",
    "Avnue": "Avenue",
    "avenue": "Avenue",
    "Pkwy": "Parkway",
    "ter": "Terrace",
    "Ln": "Lane"
    }

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m is None:
        return name
    if m.group() in mapping:
        name = name.replace(m.group(), mapping[m.group()])
        return name
    else:
        return name

for street_type, ways in street_types.items():
    for name in ways:
        better_name = update_name(name, mapping)
        print (name, "=>", better_name)

firshade ter => firshade Terrace
Bridgetown Rd => Bridgetown Road
Salem Rd => Salem Road
Hamilton avenue => Hamilton Avenue
Dixie Highway => Dixie Highway
Chickasaw St => Chickasaw Street
Clark St => Clark Street
E 7th St => E 7th Street
W Fifth St => W Fifth Street
Greenup St => Greenup Street
215 Calhoun St => 215 Calhoun Street
Vine St => Vine Street
Fox Cub Ln => Fox Cub Lane
Kenard Ave => Kenard Avenue
Erie Ave => Erie Avenue
Whetsel Ave => Whetsel Avenue
W Clifton Ave => W Clifton Avenue
Harrison Ave => Harrison Avenue
Valley Plaza Pkwy => Valley Plaza Parkway
Procter & Gamble Plaza => Procter & Gamble Plaza
Mount Adams Circle => Mount Adams Circle
Liberty Hill => Liberty Hill
Ritchie State => Ritchie State
Warner => Warner
Montgomery Rd. => Montgomery Road
North Bend Crossing => North Bend Crossing
508 => 508
South Fort Thomas Avnue => South Fort Thomas Avenue
Ludlow => Ludlow
Bremen Pass => Bremen Pass


The above output demonstrates how the code would update some street names using the mapping.

## 2. Clean the dataset and write the cleaned data to a file

Now, I move on to the step of actually writing our data to a csv file. To do this, I first design a schema for our data. This is structure in which the data would be stored in the csv file and this is how it would be imported to a database.

In [ ]:
db_schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

Here, I iteratively parse the XML file to extract the necessary data, update it and write it to a csv file.

In [ ]:
import csv
import codecs
import cerberus
import schema
import pprint

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields = NODE_FIELDS, way_attr_fields = WAY_FIELDS,
                  problem_chars = problemchars, default_tag_type = 'regular'):
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for field in NODE_FIELDS:
            node_attribs[field] = element.attrib[field]
      
        for tag in element.iter('tag'):
            
            tag_dict = {}
            
            # Updating the street name using the above declared mapping function if the tag is a street name
            if is_street_name(tag):
                tag_dict['value'] = update_name(tag.attrib['v'], mapping)
            else:
                tag_dict['value'] = tag.attrib['v']
            
            tag_dict['id'] = element.attrib['id']
            if problemchars.match(tag.attrib['k']):
                pass
            elif ':' in tag.attrib['k']:
                tag_dict['type'] = tag.attrib['k'].split(':')[0]
                tag_dict['key'] = tag.attrib['k'].split(':', 1)[1]
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = tag.attrib['k']

            tags.append(tag_dict)

        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':
        for field in WAY_FIELDS:
            way_attribs[field] = element.attrib[field]

        count = 0
        
        for nd in element.iter('nd'):
            nd_dict = {}
            nd_dict['id'] = element.attrib['id']
            nd_dict['node_id'] = nd.attrib['ref']
            nd_dict['position'] = count
            count += 1
            way_nodes.append(nd_dict)

        for tag in element.iter('tag'):
            tag_dict = {}
            # Updating the street name using the above declared mapping function if the tag is a street name
            if is_street_name(tag):
                tag_dict['value'] = update_name(tag.attrib['v'], mapping)
            else:
                tag_dict['value'] = tag.attrib['v']

            tag_dict['id'] = element.attrib['id']

            if problemchars.match(tag.attrib['k']):
                pass
            elif ':' in tag.attrib['k']:
                tag_dict['type'] = tag.attrib['k'].split(':')[0]
                tag_dict['key'] = tag.attrib['k'].split(':', 1)[1]
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = tag.attrib['k']

            tags.append(tag_dict)
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
    
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=db_schema):
    """Raise ValidationError if element does not match schema"""
    # print(element)
    if validator.validate(element, schema) is not True:
        for field, errors in validator.errors.items():
            message_string = "\nElement of type '{0}' has the following errors:\n{1}"
            error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: v for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', encoding = 'utf8') as nodes_file, \
    codecs.open(NODE_TAGS_PATH, 'w', encoding = 'utf8') as nodes_tags_file, \
    codecs.open(WAYS_PATH, 'w', encoding = 'utf8') as ways_file, \
    codecs.open(WAY_NODES_PATH, 'w', encoding = 'utf8') as way_nodes_file, \
    codecs.open(WAY_TAGS_PATH, 'w', encoding = 'utf8') as way_tags_file:
        
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

process_map(filename, validate = True)

These are the files obtained and their respective sizes, after the above process completes:

1. nodes.csv         - 74,981 KB
2. nodes_tags.csv    - 1,456 KB
3. ways.csv          - 8,450 KB
4. ways_nodes.csv    - 26,727 KB
5. ways_tags.csv     - 10,742 KB

## 3. Use the files created to load the data into a SQL Server Database

For the next phase, I will be using SQL Server to create a database of my extracted data. I start off by individually importing the csv files into the database using SQL Server Import Export Wizard.

While importing each file I describe the table schema of the data being imported. Here are the table creation queries that I used during import:

## 4. Extract insights from the database

a. Total number of unique users that have contributed to the data:

SELECT COUNT(distinct c.uid) AS 'User Count'
<br>
FROM (SELECT a.uid FROM nodes a UNION ALL SELECT b.uid FROM ways b) c;
<br>
<br>
User Count
<br>
510

b. Total number of nodes:

SELECT COUNT(DISTINCT id) AS 'Nodes Count' FROM nodes;
<br>
<br>
Nodes Count
<br>
911557

c. total number of ways:

SELECT COUNT(DISTINCT id) AS 'Ways Count' FROM ways;
<br>
<br>
Ways Count
<br>
144078

d. Top 10 contributing users and their respective contributions:

SELECT TOP 10 user_name AS 'User', COUNT(id) AS Contributions
<br>
FROM nodes
<br>
GROUP BY user_name
<br>
ORDER BY contributions DESC;

In [ ]:
%%html
<style>
  table {float: left;}
</style>

|User	|Contributions|
|:-----:|:-----------:|
|Minh Nguyen	|355011|
|lrhill	|191508|
|Nate_Wessel	|161953|
|woodpeck_fixbot	|106665|
|errorcode	|12773|
|reportingsjr	|12260|
|Matt Currie	|10153|
|MichaelGSmith	|9138|
|gmensch	|7384|
|lightbulbsrwarm	|5803|

e. Finding the religions that have places of worship in the city

SELECT DISTINCT value AS 'Religion', COUNT(*) AS 'Number of Places of Worship'
<br>
FROM nodes_tags
<br>
WHERE id IN (
<br>
	SELECT DISTINCT id
    <br>
	FROM nodes_tags
    <br>
	WHERE value = 'place_of_worship'
    <br>
	) AND tag_key = 'religion' GROUP BY value;

In [ ]:
%%html
<style>
  table {float: left;}
</style>

|Religion|Number of Places of Worship|
|:----:|:------:|
|christian|	318|
|jewish|	4|
|muslim|	1|

As one can see from the above data, majority of the places of worship are Christian and only a handful of other places.

f. Finding the 10 most referenced nodes in ways

SELECT TOP 10 a.value AS Value, a.tag_key AS 'Tag Key', COUNT(b.node_id) AS Count
<br>
FROM nodes_tags a, ways_nodes b
<br>
WHERE a.id = b.node_id 
<br>
GROUP BY a.value, a.tag_key
<br>
ORDER BY Count DESC;

In [ ]:
%%html
<style>
  table {float: left;}
</style>

|Value	|Tag Key|	Count|
|:----:|:----:|:----:|
|crossing|	highway|	11268|
|traffic_signals|	highway|	5416|
|turning_circle|	highway|	4016|
|level_crossing|	railway|	688|
|stop|	highway|	667|
|tower|	power|	617|
|zebra|	crossing|	498|
|motorway_junction|	highway|	418|
|gate|	barrier|	338|
|yes|	crossing|	143|

## Other ideas about the dataset

The data is somewhat incomplete to get a wholistic idea of the city. A good improvement to the data collection process would be setting up a set of standards to follow while adding new nodes. The data can be more descreptive wherein the user can add more information about the node to the OSM data. One simple improvement could be adding the start and close date for a node.

For exeample, a node could have had a restaurant at that place from the year 2010 to 2014. Later, the restaurant closed and the spot was taken up by a Courier service like Fedex from 2015. In such cases, we might have two different nodes that point to a same location on the map, which can lead to ambiguity. If we add a start date and close date column, we can different between the two establishments that occupied the spot by the dates. So a query to access the node could return two values where a combined primary key of node and start date will allow us differentiate the establishments.

Benefits of the Change:
On a longer timeline, this change will allow us to identify how a particular city has changed over time. For example, the birth of trains saw the development of a train station in the city around which other businesses developed.

Anticipated Problems in Implementing the Change:
A user who is adding the node will need to have some prior knowledge about it. In the current scenario, any user can submit information about a location and he/she need not worry about timeline specific details. For example, a tourist visiting Cincinnati for a week can submit info about a location. But if we add timeline based details, he won't be able to submit this data. One way to tackle this problem is to keep the start_date and end_date columns optional, which will allow everyone to submit their inputs.

## References

1. https://docs.python.org/3.3/
2. https://docs.microsoft.com/en-us/sql/